In [2]:
from pyresparser import ResumeParser
import os
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    UnexpectedAlertPresentException,
    NoSuchElementException,
)
from webdriver_manager.chrome import ChromeDriverManager


import csv


In [8]:

class WhatsApp(object):
    def __init__(self, browser=None):
        self.BASE_URL = "https://web.whatsapp.com/"
        # self.suffix_link = "https://wa.me/"
        self.suffix_link = "https://web.whatsapp.com/send?phone="

        if not browser:
            browser = webdriver.Chrome(
                ChromeDriverManager().install(),
                options=self.chrome_options,
            )

        self.browser = browser

        self.wait = WebDriverWait(self.browser, 600)
        self.login()
        self.mobile = ""

    @property
    def chrome_options(self):
        chrome_options = Options()
        if sys.platform == "win32":
            chrome_options.add_argument("--profile-directory=Default")
            chrome_options.add_argument("--user-data-dir=C:/Temp/ChromeProfile")
        else:
            chrome_options.add_argument("start-maximized")
            chrome_options.add_argument("--user-data-dir=./User_Data")
        return chrome_options

    def login(self):
        self.browser.get(self.BASE_URL)
        self.browser.maximize_window()

    def logout(self):
        prefix = "//div[@id='side']/header/div[2]/div/span/div[3]"
        dots_button = self.wait.until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                    f"{prefix}/div[@role='button']",
                )
            )
        )
        dots_button.click()

        logout_item = self.wait.until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                    f"{prefix}/span/div[1]/ul/li[last()]/div[@role='button']",
                )
            )
        )
        logout_item.click()

    def get_phone_link(self, mobile) -> str:

        return f"{self.suffix_link}{mobile}"

    def catch_alert(self, seconds=3):

        try:
            WebDriverWait(self.browser, seconds).until(EC.alert_is_present())
            alert = self.browser.switch_to_alert.accept()
            return True
        except Exception as e:
            print(e)
            return False

    def find_user(self, mobile) -> None:

        try:
            self.mobile = mobile
            link = self.get_phone_link(mobile)
            self.browser.get(link)
            # return
            self.catch_alert()
            return
            action_button = self.wait.until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="action-button"]'))
            )
            action_button.click()
            time.sleep(2)
            go_to_web = self.wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="fallback_block"]/div/div/a')
                )
            )
            go_to_web.click()
            time.sleep(1)
            return
        except UnexpectedAlertPresentException as bug:
            print(bug)
            time.sleep(1)
            self.find_user(mobile)

    def find_by_username(self, username):

        try:
            search_box = self.wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="side"]/div[1]/div/label/div/div[2]')
                )
            )
            search_box.clear()
            search_box.send_keys(username)
            search_box.send_keys(Keys.ENTER)
        except Exception as bug:
            error = f"Exception raised while finding user {username}\n{bug}"
            print(error)

    def username_exists(self, username):

        try:
            search_box = self.wait.until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="side"]/div[1]/div/label/div/div[2]')
                )
            )
            search_box.clear()
            search_box.send_keys(username)
            search_box.send_keys(Keys.ENTER)
            opened_chat = self.browser.find_element_by_xpath(
                "/html/body/div/div[1]/div[1]/div[4]/div[1]/header/div[2]/div[1]/div/span"
            )
            title = opened_chat.get_attribute("title")
            if title.upper() == username.upper():
                return True
            else:
                return False
        except Exception as bug:
            error = f"Exception raised while finding user {username}\n{bug}"
            print(error)

    def send_message(self, message):

        try:
            inp_xpath = (
                '//*[@id="main"]/footer/div[1]/div/span[2]/div/div[2]/div[1]/div/div[2]'
            )
           
            input_box = self.wait.until(
                EC.presence_of_element_located((By.XPATH, inp_xpath))
            )
            print(input_box)
            input_box.send_keys(message + Keys.ENTER)
            return 
            print(f"Message sent successfuly to {self.mobile}")
            self.browser.get("https://web.whatsapp.com/")
            self.catch_alert()
            return
            
        except (NoSuchElementException, Exception) as bug:
            print(bug)
            print(f"Failed to send a message to {self.mobile}")

        finally:
            print("send_message() finished running ")

    def find_attachment(self):
        clipButton = self.wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="main"]/footer//*[@data-icon="clip"]/..')
            )
        )
        clipButton.click()

    def send_attachment(self):
        # Waiting for the pending clock icon to disappear
        self.wait.until_not(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="main"]//*[@data-icon="msg-time"]')
            )
        )

        sendButton = self.wait.until(
            EC.presence_of_element_located(
                (
                    By.XPATH,
                    '//*[@id="app"]/div[1]/div[1]/div[2]/div[2]/span/div[1]/span/div[1]/div/div[2]/div/div[2]/div[2]/div/div/span',
                )
            )
        )
        sendButton.click()

    def send_picture(self, picture):

        try:
            filename = os.path.realpath(picture)
            self.find_attachment()
            # To send an Image
            imgButton = self.wait.until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//*[@id="main"]/footer//*[@data-icon="attach-image"]/../input',
                    )
                )
            )
            imgButton.send_keys(filename)
            self.send_attachment()
            print(f"Picture has been successfully sent to {self.mobile}")
        except (NoSuchElementException, Exception) as bug:
            print(bug)
            print(f"Failed to send a picture to {self.mobile}")

        finally:
            print("send_picture() finished running ")

    def send_video(self, video):

        try:
            filename = os.path.realpath(video)
            self.find_attachment()
            # To send a Video
            video_button = self.wait.until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//*[@id="main"]/footer//*[@data-icon="attach-image"]/../input',
                    )
                )
            )
            video_button.send_keys(filename)
            self.send_attachment()
            print(f"Video has been successfully sent to {self.mobile}")
        except (NoSuchElementException, Exception) as bug:
            print(bug)
            print(f"Failed to send a video to {self.mobile}")
        finally:
            print("send_video() finished running ")

    def send_file(self, filename):

        try:
            filename = os.path.realpath(filename)
            self.find_attachment()
            document_button = self.wait.until(
                EC.presence_of_element_located(
                    (
                        By.XPATH,
                        '//*[@id="main"]/footer//*[@data-icon="attach-document"]/../input',
                    )
                )
            )
            document_button.send_keys(filename)
            self.send_attachment()
        except (NoSuchElementException, Exception) as bug:
            print(bug)
            print(f"Failed to send a PDF to {self.mobile}")
        finally:
            print("send_file() finished running ")


In [70]:
main_dir = '/Users/b0224754/Documents/Tej-Contact/Resume'

resume_path = f'./vishal_jangid.pdf'
data = ResumeParser(resume_path).get_extracted_data()
# print(data['mobile_number'])
# print(data)


/Users/b0224754/Documents/Tej-Contact/env/lib/python3.8/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.1 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [14]:
file_path = '/Users/b0224754/Documents/python/sample_data.csv'

rows = []
with open(file_path, 'r') as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        rows.append(row)
# print(rows)



In [15]:
def create_msg(name, hr_name, script_msg, num_script_var, var_values):

    script_msg = [msg.replace('CANDIDATE NAME', name) for msg in script_msg]
    script_msg = [msg.replace('HR NAME', hr_name) for msg in script_msg]
    a = ''
    for i in range(num_script_var):
        a = i + 1
        script_msg = [msg.replace(str(a) + '*', var_values[i]) for msg in script_msg]
    return script_msg

    # print(name, script_msg, num_script_var, var_values)
    

In [16]:
msg = WhatsApp()

hr_name = rows[0][1]

script_msg = []
script_msg = [i for i in rows[1]]
script_msg = script_msg[1:]
script_msg

num_script_var = int(rows[2][1])

data = []
for i in range(4, len(rows)):
    candidate_name = rows[i][0]
    number = '91'+rows[i][1]
    status = rows[i][2]
    # print(rows[i])
    if status == 'No':
        Meessages = create_msg(candidate_name, hr_name, script_msg, num_script_var, rows[i][3:])
        
        msg.find_user(number)
        for message in Meessages:
            msg.send_message(message)
            msg.catch_alert()





====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/b0224754/.wdm/drivers/chromedriver/mac64_m1/99.0.4844.51/chromedriver] found in cache
/var/folders/6c/0ptyz43n0q17frnrc345886h0000gn/T/ipykernel_30130/2573341182.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(


Message: 

<selenium.webdriver.remote.webelement.WebElement (session="9f79639810aa588786fd3f460353f648", element="374d5784-c0c5-41ae-a0cd-cd6738fadf8f")>
send_message() finished running 
Message: 

<selenium.webdriver.remote.webelement.WebElement (session="9f79639810aa588786fd3f460353f648", element="374d5784-c0c5-41ae-a0cd-cd6738fadf8f")>
send_message() finished running 
Message: 

<selenium.webdriver.remote.webelement.WebElement (session="9f79639810aa588786fd3f460353f648", element="374d5784-c0c5-41ae-a0cd-cd6738fadf8f")>
send_message() finished running 
Message: 

<selenium.webdriver.remote.webelement.WebElement (session="9f79639810aa588786fd3f460353f648", element="374d5784-c0c5-41ae-a0cd-cd6738fadf8f")>
send_message() finished running 
Message: 

<selenium.webdriver.remote.webelement.WebElement (session="9f79639810aa588786fd3f460353f648", element="374d5784-c0c5-41ae-a0cd-cd6738fadf8f")>
send_message() finished running 
Message: 

<selenium.webdriver.remote.webelement.WebElement (sess

In [ ]:

msg = WhatsApp()

numbers = ["919811122581", "919612576467"]

for number in numbers:
    msg.find_user(number)
    for i in range(5):

        msg.send_message("This is bot")
        msg.catch_alert()

    # msg.send_message("bot send")

In [ ]:
msg = 'This is a test message from bot'
users = ["+91 9602614291"]
auto_whatsapp.sendChat(users, msg)

In [2]:
from tkinter import * 
from tkinter.ttk import *

In [2]:
'''
author : abhayparashar31
This Program Take Height(CM) and Weight(KG) and Returns The BMI(Body Mass Index) Value As Output In a Pop up Box.
'''
from tkinter import *
from tkinter import messagebox
def get_height():
    height = float(ENTRY2.get())
    return height
def get_weight():
    weight = float(ENTRY1.get())
    return weight
def calculate_bmi(): 
    try:
        height = get_height()
        weight = get_weight()
        height = height / 100.0
        bmi = weight / (height ** 2)
    except ZeroDivisionError:
        messagebox.showinfo("Result", "Please enter positive height!!")
    except ValueError:
        messagebox.showinfo("Result", "Please enter valid data!")
    else:
        messagebox.showinfo("Your BMI Calculated is : ", bmi)

if __name__ == '__main__':
    TOP = Tk()
    TOP.bind("<Return>", calculate_bmi)
    TOP.geometry("400x400")
    TOP.configure(background="#8c52ff")
    TOP.title("BMI Calculator")
    TOP.resizable(width=False, height=False)
    LABLE = Label(TOP, bg="#8c52ff",fg="#ffffff", text="Welcome to BMI Calculator", font=("Helvetica", 15, "bold"), pady=10)
    LABLE.place(x=55, y=0)
    LABLE1 = Label(TOP, bg="#ffffff", text="Enter Weight (in kg):", bd=6,
                   font=("Helvetica", 10, "bold"), pady=5)
    LABLE1.place(x=55, y=60)
    ENTRY1 = Entry(TOP, bd=8, width=10, font="Roboto 11")
    ENTRY1.place(x=240, y=60)
    LABLE2 = Label(TOP, bg="#ffffff", text="Enter Height (in cm):", bd=6,
                   font=("Helvetica", 10, "bold"), pady=5)
    LABLE2.place(x=55, y=121)
    ENTRY2 = Entry(TOP, bd=8, width=10, font="Roboto 11")
    ENTRY2.place(x=240, y=121)
    BUTTON = Button(bg="#000000",fg='#ffffff', bd=12, text="BMI", padx=33, pady=10, command=calculate_bmi,
                    font=("Helvetica", 20, "bold"))
    BUTTON.grid(row=5, column=0, sticky=W)
    BUTTON.place(x=115, y=250)
    TOP.mainloop()